In [1]:
import os
import numpy as np
import pandas as pd
import corus
from tqdm import tqdm

np.random.seed(42)

### 1. Load `lenta-ru-news`

In [2]:
DATA_SIZE = 100_000 # cut off size for faster performance

path = 'lenta-ru-news.csv.gz'

if not os.path.exists(path):
    !wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz
else:
    print("Already loaded.")

Already loaded.


In [3]:
records = corus.load_lenta(path)
def parse(record) -> dict[str, str]:
    return dict(title=record.title, text=record.text, topic=record.topic)
df = pd.DataFrame(list(tqdm(map(lambda r: parse(r), records), desc='load lenta')))
df = df.sample(DATA_SIZE, random_state=42)
df.head(10)

load lenta: 739351it [00:20, 36232.50it/s]


,title,text,topic
153198,EgyptAir объявила о подорожании билетов,Египетский перевозчик EgyptAir сообщил о возмо...,Путешествия
169154,Глава Красногорского района Подмосковья ушел в...,Глава Красногорского района Московской области...,Россия
83745,Милонов предложил запретить россиянам сидеть в...,Депутат Виталий Милонов внес в Госдуму законоп...,Россия
10029,Женщинам в детородном возрасте разрешили посещ...,Верховный суд Индии разрешил женщинам в фертил...,Мир
6445,Россиянам пообещали дешевый хлеб,Россиянам не стоит бояться роста цен на хлеб —...,Экономика
45187,МОК наказал Мутко,Российский вице-премьер Виталий Мутко пожизнен...,Спорт
399176,Жара резко увеличила энергопотребление в Москве,Потребление электроэнергии в Москве в последни...,Экономика
66430,Федор Емельяненко и Милонов снимутся во «Лжи М...,"На Урале снимут картину «Ложь Матильды», посвя...",Культура
78735,Трамп высмеял противившихся увольнению директо...,Президент США Дональд Трамп опубликовал подбор...,Мир
102670,В Москве подтвердили передачу Сербии шести ист...,Весной 2017 года Сербия получит шесть истребит...,Силовые структуры


### 2 Preprocessing

#### 2.1 Clear up data

In [4]:
import re
from nltk.corpus import stopwords
import pymorphy3

morph = pymorphy3.MorphAnalyzer()
russian_stopwords = set(stopwords.words('russian'))

#### Описание
1. Normalization `text = re.sub(r'[^а-яё\s]', '', text.lower())`
2. Lemmatization `lemma = morph.parse(word)[0].normal_form` 

In [5]:
init = ' '.join(df['text'].iloc[0:10])
example = init

print(f'__________Initial__________\nTokens unique count: {len(set(example.split()))}\n{example[:100]}...')
__old_tokens_cnt = len(set(example.split()))
example = re.sub(r'[^а-яё\s]', '', example.lower())
print(f'_______Normalization_______\nTokens unique count: {len(set(example.split()))}, skipped {(1 - len(set(example.split()))/__old_tokens_cnt) * 100:.1f}%\n{example[:100]}...')
__old_tokens_cnt = len(set(example.split()))
example = ' '.join(map(lambda w: morph.parse(w)[0].normal_form, example.split()))
print(f'___________Lemma___________\nTokens unique count: {len(set(example.split()))}, reduced {(1 - len(set(example.split()))/__old_tokens_cnt) * 100:.1f}%\n{example[:100]}...')
__old_tokens_cnt = len(set(example.split()))
example = ' '.join(filter(lambda w: w not in russian_stopwords, example.split()))
print(f'______Skip Stop Words______\nTokens unique count: {len(set(example.split()))}, skipped {(1 - len(set(example.split()))/__old_tokens_cnt) * 100:.1f}%\n{example[:100]}...')


__________Initial__________
Tokens unique count: 1227
Египетский перевозчик EgyptAir сообщил о возможном повышении стоимости билетов на свои международные...
_______Normalization_______
Tokens unique count: 1074, skipped 12.5%
египетский перевозчик  сообщил о возможном повышении стоимости билетов на свои международные рейсы и...
___________Lemma___________
Tokens unique count: 851, reduced 20.8%
египетский перевозчик сообщить о возможный повышение стоимость билет на свой международный рейс изз ...
______Skip Stop Words______
Tokens unique count: 792, skipped 6.9%
египетский перевозчик сообщить возможный повышение стоимость билет свой международный рейс изз девал...


Так как для задачи нам необходим смысл текста, то нам важно убрать все ненужное и сохранить смысл текста
 - Нормализация  `re.sub(r'[^а-яё\s]', '', example.lower())` позволяет привести слова к единому виду и убрать не смысловые токены
 - Лемматизация позволяет избавиться от проблемы разных форм слов и тем самым я увеличиваю количество вхождения каждого токена, увеличиваю количество связей тежду текстами через слова
 - убираю не смысловые стоп слова для ускорения обучения и повышения качества н-грамм 

In [6]:
import functools


# кэшируем ответы, чтобы препроцессить не бесконечность времени
@functools.lru_cache(maxsize=10_000)
def normalize(token):
    return morph.parse(token)[0].normal_form


def preprocess_text(text):
    text = re.sub(r'[^а-яё\s]', '', text.lower())
    return ' '.join(
        filter(
            lambda token: token not in russian_stopwords,
            map(
                normalize,
                text.split()
            )
        )
    )


assert preprocess_text(init) == example, 'processing error'

In [7]:
df['processed_text'] = list(tqdm(map(
    preprocess_text, df['text'].to_list()), desc='preprocess text', total=len(df)))

preprocess text: 100%|██████████| 100000/100000 [05:57<00:00, 279.45it/s]


##### 2.1.1 Drop not representative classes

В датасете есть нерепрезентативные классы, количество вхождений которых не позволяет качественно обучить классификатор (на этих классах), скнитем их в кучу `other`. Таким образом мы сохраняем количество данных и сохраняем изначальное распределение, что очень полезно при раскатывании модели в реальной жизни.

Из эмпирических соображений считаем, что ***минимальное число наблюдений на класс, необходимое для обучения, будет 10% от размера датасета***   

In [8]:
MIN_TOPIC_FREQ = int(.01*len(df))
assert MIN_TOPIC_FREQ > 3, 'Too small MIN_TOPIC_FREQ for split train/val/test'
print(f'Minimal topic frequency: {MIN_TOPIC_FREQ}')

Minimal topic frequency: 1000


In [9]:
print('Topics with counts before preprocessing')
df['topic'].value_counts()

Topics with counts before preprocessing


topic
Россия               21871
Мир                  18494
Экономика            10737
Спорт                 8632
Культура              7337
Наука и техника       7129
Бывший СССР           7100
Интернет и СМИ        6181
Из жизни              3718
Дом                   2891
Силовые структуры     2661
Ценности              1079
Бизнес                 967
Путешествия            855
69-я параллель         178
Крым                    82
Культпросвет            45
                        23
Легпром                 10
Библиотека               8
Сочи                     1
Оружие                   1
Name: count, dtype: int64

In [10]:
from enum import Enum


class Strategy(Enum):
    """
    Тут отразил разные стратегии для решения проблемы нерепрезентативности классов
    """
    NONE = 0  # ничего не делаем
    DROP = 1  # опускаем нерепрезентативные классы
    REPLACE_WITH_OTHER = 2  # скидываем из в кучу

In [11]:
STRATEGY = Strategy.REPLACE_WITH_OTHER

match STRATEGY:
    case Strategy.NONE:
        pass
    case Strategy.DROP:
        _counts = df['topic'].value_counts()
        _keep = (_counts[_counts > MIN_TOPIC_FREQ]).index.to_list()
        df = df[df['topic'].apply(lambda x: x in _keep)]
    case Strategy.REPLACE_WITH_OTHER:
        _counts = df['topic'].value_counts()
        _keep = (_counts[_counts > MIN_TOPIC_FREQ]).index.to_list()
        df['topic'] = df['topic'].apply(
            lambda x: 'other' if x not in _keep else x)

In [12]:
print(f'Topics with counts after preprocessing with strategy `{STRATEGY.name}`')
df['topic'].value_counts()

Topics with counts after preprocessing with strategy `REPLACE_WITH_OTHER`


topic
Россия               21871
Мир                  18494
Экономика            10737
Спорт                 8632
Культура              7337
Наука и техника       7129
Бывший СССР           7100
Интернет и СМИ        6181
Из жизни              3718
Дом                   2891
Силовые структуры     2661
other                 2170
Ценности              1079
Name: count, dtype: int64

#### 2.2 Train/Test/Val split

In [13]:
from sklearn.model_selection import train_test_split

X = df['processed_text']
y = df['topic']

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.4, stratify=y, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)

assert len(X_train) + len(X_val) + len(X_test) == len(X)
assert len(y_train) + len(y_val) + len(y_test) == len(y)
assert np.all(sorted(y.unique()) == sorted(y_train.unique()))
assert np.all(sorted(y.unique()) == sorted(y_test.unique()))
assert np.all(sorted(y.unique()) == sorted(y_val.unique()))

print(f'Train size: {len(X_train)}, {len(X_train) / len(X):.2f}%')
print(f'Val size: {len(X_val)}, {len(X_val) / len(X):.2f}%')
print(f'Test size: {len(X_test)}, {len(X_test) / len(X):.2f}%')

Train size: 60000, 0.60%
Val size: 20000, 0.20%
Test size: 20000, 0.20%


### 3. Dummy classifier

In [14]:
from sklearn.metrics import classification_report
from sklearn.dummy import DummyClassifier

dummy = DummyClassifier(strategy='most_frequent')
dummy.fit(X_train, y_train)
y_val_pred = dummy.predict(X_val)

print(f"Test score (accuracy): {dummy.score(X_val, y_val)}")
print(classification_report(y_val, y_val_pred, zero_division=0.))

Test score (accuracy): 0.2187
                   precision    recall  f1-score   support

            other       0.00      0.00      0.00       434
      Бывший СССР       0.00      0.00      0.00      1420
              Дом       0.00      0.00      0.00       578
         Из жизни       0.00      0.00      0.00       743
   Интернет и СМИ       0.00      0.00      0.00      1236
         Культура       0.00      0.00      0.00      1468
              Мир       0.00      0.00      0.00      3699
  Наука и техника       0.00      0.00      0.00      1426
           Россия       0.22      1.00      0.36      4374
Силовые структуры       0.00      0.00      0.00       532
            Спорт       0.00      0.00      0.00      1726
         Ценности       0.00      0.00      0.00       216
        Экономика       0.00      0.00      0.00      2148

         accuracy                           0.22     20000
        macro avg       0.02      0.08      0.03     20000
     weighted avg       

### 4. LogisticRegression

Тут надо обговорить, что параметр `max_features` для векторизаторов отвечает за количество токенов с максимально частотой в обучающем датасете. Ясно, что если токен встречается единицы раз, то он нам бесполезен, поэтому надо этот параметр брать не из головы, а ходьбы посмотреть, что в модель не попадают очень редкие токены. В коде ниже хочу отразить, что этот параметр отвечает здравому смыслу (тут я не говорю, что 18 - самая лучшая оценка минимального числа вхождения токена, это число выглядит неплохой начальной оценкой) 


p. s. Этот параметр  также можно подбирать как перцентиль то количеств вхождений каждого токена (например $
P_{90}$)

p. p. s. Явное ограничение количество токенов дает возможность явно ограничить вес модели, что супер полезно в реальных задачах с большими данными.   

In [15]:
from collections import Counter

MAX_FEATURES = 20_000

token_counter = Counter()
for text in X_train:
    token_counter.update(text.split())

_, min_common_token_count = token_counter.most_common(MAX_FEATURES)[-1]
assert min_common_token_count > 1, 'Too small min_common_token_count'
print(f'Minimum common token count: {min_common_token_count}')

Minimum common token count: 18


In [16]:
import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings('ignore', category=ConvergenceWarning)

#### 4.1. CountVectorizer

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import LogisticRegression

count_vectorized = Pipeline([
    ('vec', CountVectorizer(ngram_range=(1, 2), max_features=20_000)),
    ('scaler', MaxAbsScaler()),
    ('clf', LogisticRegression(solver='saga', tol=1e-3, max_iter=100, class_weight='balanced', random_state=42))
])
count_vectorized.fit(X_train, y_train)
y_val_pred = count_vectorized.predict(X_val)
print(classification_report(y_val, y_val_pred, zero_division=0.))

                   precision    recall  f1-score   support

            other       0.54      0.50      0.52       434
      Бывший СССР       0.79      0.80      0.79      1420
              Дом       0.82      0.85      0.83       578
         Из жизни       0.55      0.69      0.61       743
   Интернет и СМИ       0.73      0.70      0.72      1236
         Культура       0.86      0.88      0.87      1468
              Мир       0.79      0.78      0.79      3699
  Наука и техника       0.81      0.83      0.82      1426
           Россия       0.81      0.75      0.78      4374
Силовые структуры       0.55      0.63      0.59       532
            Спорт       0.96      0.96      0.96      1726
         Ценности       0.85      0.83      0.84       216
        Экономика       0.81      0.84      0.83      2148

         accuracy                           0.79     20000
        macro avg       0.76      0.77      0.76     20000
     weighted avg       0.80      0.79      0.79     2

На данном этапе подобрал параметры так, чтобы получить базовое качество архитектуры
 - **CountVectorizer**
   - `ngram_range=(1, 2)`: берем юниграммы и биграммы, дабы задействовать информацию о контексте слова
   - `max_features=20_000`: описал выше  
 - **LogisticRegression**
   - `solver='saga'`: имперически луше работает с разряженными данными и поддерживает мульти класс
   - `tol=1e-3, max_iter=100`: параметры не, которые не влияют на ход обучения, только ставят ограничения на количество итераций, подобраны так, чтоб не было `ConvergenceWarning: The max_iter was reached which means the coef_ did not converge` Так как у нас данные разряженные, то это может влиять на сходимость задачи, по этому я явно ослабил требование "хорошо обученной" модели в контексте ошибки на обучающей выборке (параметром `tol`) и сохранил дефолтное значение `max_iter` (до максимума все равно не дошло обучение)  
    - `class_weight='balanced'`: так как в данных разница в частотности классов доходит до порядков, то имеет смысл сбалансировать веса классов 

p. s. `MaxAbsScaler` не меняет форму распределения и сохраняет нули (и разрежённость соответственно), однако полезен для логистической регрессии, дабы улучшить численную сходимость задачи. Можно было использовать, например, l1/l2 нормализацию, однако такая нормализация меняет форму распределения, а мы в домашке хотим посмотреть на перфоманс чистых векторизаторов (как я понял)

#### 4.1. TfIdfVectorizer

In [18]:
tfidf_vectorized = Pipeline([
    ('vec', TfidfVectorizer(ngram_range=(1, 2), max_features=20_000)),
    ('clf', LogisticRegression(solver='saga', tol=1e-3, max_iter=100, class_weight='balanced', random_state=42))
])
tfidf_vectorized.fit(X_train, y_train)
y_val_pred = tfidf_vectorized.predict(X_val)
print(classification_report(y_val, y_val_pred, zero_division=0.))

                   precision    recall  f1-score   support

            other       0.42      0.63      0.50       434
      Бывший СССР       0.79      0.86      0.82      1420
              Дом       0.75      0.88      0.81       578
         Из жизни       0.53      0.75      0.62       743
   Интернет и СМИ       0.73      0.73      0.73      1236
         Культура       0.86      0.88      0.87      1468
              Мир       0.82      0.78      0.80      3699
  Наука и техника       0.82      0.83      0.82      1426
           Россия       0.85      0.72      0.78      4374
Силовые структуры       0.48      0.71      0.57       532
            Спорт       0.95      0.96      0.96      1726
         Ценности       0.97      0.56      0.71       216
        Экономика       0.85      0.82      0.84      2148

         accuracy                           0.79     20000
        macro avg       0.76      0.78      0.76     20000
     weighted avg       0.81      0.79      0.80     2

По результатам можно понять
1. Метрики TfIdf не сиольно лучше, чем Count. Это может говорить о том, что idf балансировка не дает качества. Можно предположить, что данные и так достаточно сбалансированы.
2. f1 на большистве классов высокое для обоих подходов, однако есть 3 класса с маленьким весом, на которы качество ниже. При подборе гипперпараметром можно попробовать class_weight='balanced' для одинакого веса каждого класса

### 5. Search

In [19]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import BaseCrossValidator


params = {
    'clf__C': [1e-1, 1], # шатаем l2 регуляризацию
    'clf__penalty': ['l2', 'elasticnet'], # шатаем l2 регуляризацию
    'clf__class_weight': ['balanced', None]
}

class CustomSplit(BaseCrossValidator):
    def __init__(self, X_train, X_test, y_train, y_test):
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        
    def split(self, X=None, y=None, groups=None):
        yield (np.arange(len(self.X_train)), np.arange(len(self.X_test)))
        
    def get_n_splits(self, X=None, y=None, groups=None):
        return 1

custom_split = CustomSplit(X_train, X_test, y_train, y_test)

grid = GridSearchCV(
    tfidf_vectorized,
    params,
    cv=custom_split,
    n_jobs=-1,
    verbose=1
)
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    grid.fit(pd.concat((X_train, X_test)), pd.concat((y_train, y_test)))

print(f'Best params: {grid.best_params_}')
print(f'Best CV score: {grid.best_score_:.3f}')

Fitting 1 folds for each of 8 candidates, totalling 8 fits


/home/user/education/dl_nlp/.venv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Best params: {'clf__C': 1, 'clf__class_weight': None, 'clf__penalty': 'l2'}
Best CV score: 0.871


In [20]:
best_model = grid.best_estimator_
y_val_pred = best_model.predict(X_val)
print(classification_report(y_val, y_val_pred, zero_division=0.))

                   precision    recall  f1-score   support

            other       0.74      0.35      0.48       434
      Бывший СССР       0.82      0.81      0.82      1420
              Дом       0.86      0.80      0.83       578
         Из жизни       0.70      0.54      0.61       743
   Интернет и СМИ       0.78      0.69      0.73      1236
         Культура       0.87      0.89      0.88      1468
              Мир       0.78      0.84      0.81      3699
  Наука и техника       0.83      0.85      0.84      1426
           Россия       0.78      0.84      0.81      4374
Силовые структуры       0.72      0.40      0.52       532
            Спорт       0.96      0.96      0.96      1726
         Ценности       0.93      0.74      0.82       216
        Экономика       0.82      0.86      0.84      2148

         accuracy                           0.81     20000
        macro avg       0.81      0.74      0.76     20000
     weighted avg       0.81      0.81      0.81     2

Получаем не сильный прирост, что может быть связано с малым `max_iter` (но не хочеться ждать 1000 лет) + сложностями численной оптимизации sparsed векторов. Как дополнение, можно использовать l2 Normalizer.